In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --quiet -U langchain langchain_openai langgraph langchainhub langchain_experimental faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.8/120.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00


***
1.   migrate to langgraph
2.   add reviewer
3.   split retrieval funcitons to people, project agents
4.   add viz
***



## **My Tries**

In [ ]:
import openai
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, AgentExecutor, Tool, ZeroShotAgent
from langchain.memory import ConversationBufferMemory
import faiss
import pandas as pd
import numpy as np
import os

# Set OpenAI API key
# openai_api = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Hackathon/openai_api.xlsx').iloc[0,0]
# openai.api_key = openai_api


In [ ]:

import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_PROJECT"] = "LangGraph Multi-agent TT"

In [ ]:
import pandas as pd

# Paths to files in Google Drive
people_file = '/content/drive/MyDrive/Colab Notebooks/Hackathon/Employees.xlsx'
projects_file = '/content/drive/MyDrive/Colab Notebooks/Hackathon/project_descriptions.csv'
allocation_file = '/content/drive/MyDrive/Colab Notebooks/Hackathon/allocation_table.xlsx'

# Load data
people_data = pd.read_excel(people_file)
project_data = pd.read_csv(projects_file)
initial_allocation = pd.read_excel(allocation_file)
project_data.rename(columns={'Project Name':'name'}, inplace=True)
people_data

,Name,Role,Skills,Vacation Time,PiT Capacity,Performance Review,Communication Preference,Management Advice,1on1s
0,Judy,Quantitative Analyst AVP,"Machine Learning\nPython, C++\nFinancial Modeling",1/1/2025 - 2/1/2025\n6/1/2025-6/7/2025,Yes,Outstanding,English and Korean,She is good at managing projects with abundant...,Yes
1,Alice,Quantitative Analyst VP,"Machine Learning\nPython, R\nFinancial Modeling",5/1/2025 - 6/1/2025\n8/1/2025-8/15/2025,No till 2/1/2025,Outstanding,English,Now busy on an important project,Yes
2,Susan,Quantitative Analyst AVP,"Accounting\nPython, R\nFinancial Modeling",1/15/2025 - 2/15/2025\n3/18/2025-3/27/2025,Yes,Good,English and Chinese,She is a technical person with accounting expe...,Yes
3,Tom,Quantitative Analyst AVP,Equity Research\nPython\nFinancial Modeling,1/15/2025 - 2/15/2025\n11/1/2025-11/7/2025,Yes,Outstanding,English and Chinese,He is the best friend with Susan,Yes
4,Jack,Financial Analyst AVP,Python\nFinancial Modeling\nAccounting,4/1/2025 - 6/1/2025,Yes,Good,English,Very new to the team,Yes
5,Kyle,Financial Analyst VP,"Accounting\nPython, R\nFinancial Modeling",\n6/1/2025-6/7/2025,No till 3/1/2025,Outstanding,English,Many years of experience senior VP,Yes
6,Alex,Software Development Engineer VP,"Computer Science\nPython, C++",6/1/2025 - 7/1/2025\n11/1/2025-11/15/2025,Yes,Outstanding,English,Senior SDE in the team,Yes
7,Maria,Software Development Engineer AVP,"Computer Science\nPython, C++",1/1/2025 - 2/1/2025\n6/1/2025-6/7/2025,Yes,Good,English and Japanese,Junior SDE also speaking Japanese,Yes
8,Jane,Software Development Engineer AVP,"Computer Science\nPython, C++",1/15/2025 - 2/15/2025\n3/18/2025-3/27/2025,Yes,Outstanding,English,Very active and responsive,Yes
9,Christina,Human Resource Analyst,Project Management\nCommunication,5/1/2025 - 6/1/2025\n8/1/2025-8/15/2025,Yes,Outstanding,English,Good presentation skills,Yes


In [ ]:
project_data

,name,Description
0,**Project 1: Equity Valuation Model**,Developed a comprehensive equity valuation mod...
1,**Project 2: Algorithmic Trading Bot**,Engineered an algorithmic trading bot using Py...
2,**Project 3: Sentiment Analysis for Stock Pred...,Created a sentiment analysis tool that scraped...
3,**Project 4: Portfolio Optimization Dashboard**,Designed an interactive dashboard in R Shiny f...
4,**Project 5: Risk Management Simulation**,Developed a Monte Carlo simulation tool to ass...


In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# Initialize embedding model
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# Generate combined descriptions for each person
people_descriptions = (people_data['Name'] + " is a " + people_data['Role'] + " with skills in " + people_data['Skills']).tolist()
project_descriptions = (project_data['name'] + ": " + project_data['Description']).tolist()
# Generate embeddings
person_embeddings = embeddings.embed_documents(people_descriptions)
project_embeddings = embeddings.embed_documents(project_descriptions)
# Initialize FAISS index and add embeddings
dimension = len(person_embeddings[0])  # Dimensionality of embeddings
project_index = faiss.IndexFlatL2(dimension)
person_index = faiss.IndexFlatL2(dimension)

project_index.add(np.array(project_embeddings))
person_index.add(np.array(person_embeddings))

faiss.write_index(project_index, "project_index.faiss")
faiss.write_index(person_index, "person_index.faiss")


# Save the FAISS index to Google Drive
# faiss.write_index(faiss_index, '/content/drive/MyDrive/Colab Notebooks/Hackathon/faiss_people_index.index')


In [ ]:
import faiss

faiss_index = faiss.read_index('/content/drive/MyDrive/Colab Notebooks/Hackathon/faiss_people_index.index')

In [ ]:
# Initialize availability and capacity per quarter for each person
quarters = ['Q1', 'Q2', 'Q3', 'Q4']

# Assume full availability and capacity initially
for quarter in quarters:
    people_data[f'{quarter}_available'] = True
    people_data[f'{quarter}_capacity'] = people_data['PiT Capacity']

# Adjust availability based on vacation time
def adjust_availability(row):
    if pd.notnull(row['Vacation Time']):
        vacation_times = str(row['Vacation Time']).split(',')
        for vt in vacation_times:
            vt = vt.strip()
            if vt in quarters:
                row[f'{vt}_available'] = False
    return row

people_data = people_data.apply(adjust_availability, axis=1)

In [82]:
# Prepare combined project data
projects_info = []
for _, project in project_data.iterrows():
    project_info = {
        'name': project['name'],
        'description': project['Description'],
        'constraints': project.get('constraints', 'should be at least 2 candidates')
    }
    projects_info.append(project_info)

# Prepare combined candidate data
candidates_info = []
for idx, person_row in people_data.iterrows():
    candidate = {
        "Name": person_row['Name'],
        "Role": person_row['Role'],
        "Skills": person_row['Skills'],
        "Performance Review": person_row['Performance Review'],
        "Capacity": {quarter: person_row[f'{quarter}_capacity'] for quarter in quarters},
        "Availability": {quarter: person_row[f'{quarter}_available'] for quarter in quarters}
    }
    candidates_info.append(candidate)


In [83]:
# Function to format projects for the prompt
def format_projects(projects):
    projects_str = ""
    for project in projects:
        projects_str += f"Project Name: {project['name']}\nDescription: {project['description']}\nConstraints: {project['constraints']}\n"
    return projects_str

# Function to format candidates for the prompt
def format_all_candidates(candidates):
    candidates_str = ""
    for candidate in candidates:
        capacities = ', '.join([f"{q}: {candidate['Capacity'][q]}" for q in quarters])
        availabilities = ', '.join([f"{q}: {candidate['Availability'][q]}" for q in quarters])
        candidates_str += f"Name: {candidate['Name']}, Role: {candidate['Role']}, Skills: {candidate['Skills']}, Performance Review: {candidate['Performance Review']}, Capacities: {capacities}, Availabilities: {availabilities}\n"
    return candidates_str

# Prepare the formatted strings
projects_str = format_projects(projects_info)
candidates_str = format_all_candidates(candidates_info)


In [60]:
projects_str

'Project Name: **Project 1: Equity Valuation Model**\nDescription: Developed a comprehensive equity valuation model using discounted cash flow (DCF) analysis for a portfolio of tech stocks, which provided insights into fair value estimates. Leveraged advanced Excel functionality and Python for data analysis, resulting in a 15% improved accuracy in predicting stock price movements. The project showcased my skills in financial modeling, data visualization, and quantitative analysis.\nConstraints: should be at least 2 candidates\nProject Name: **Project 2: Algorithmic Trading Bot**\nDescription: Engineered an algorithmic trading bot using Python and the Alpaca API, designed to execute trades based on quantitative signals derived from historical price data. The bot achieved a 20% return on investment over six months, outperforming the S&P 500. This project honed my skills in machine learning, backtesting frameworks, and real-time data processing.\nConstraints: should be at least 2 candidat

In [88]:
# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=openai.api_key)


# Adjust the assignment prompt template
assignment_prompt_template = PromptTemplate(
    input_variables=["projects", "candidates"],
    template="""
You are the Assignment Agent responsible for assigning candidates to projects across all quarters.

Projects:
{projects}

Candidates:
{candidates}

Constraints:
- Ensure that each candidate is not assigned beyond their capacity for each quarter.
- Candidates must be available in the quarter they are assigned.
- Match candidate skills and roles with project requirements.
- Consider performance reviews when selecting candidates.
- Each quarter should have at least 2 candidates.

Your task is to assign the best candidates to each project for each quarter.

Provide the assignments in the following JSON format, and output only the JSON:

[
    {{
        "Project": "Project Name",
        "Assignments": {{
            "Q1": ["Candidate Name 1", "Candidate Name 2"],
            "Q2": ["Candidate Name 3"],
            "Q3": ["Candidate Name 1", "Candidate Name 2"],
            "Q4": ["Candidate Name 4"]
        }}
    }},
    ...
]

Ensure the JSON is valid and parsable. Do not include any text before or after the JSON output.
"""
)

In [89]:
# Initialize the assignment chain
assignment_chain = LLMChain(prompt=assignment_prompt_template, llm=llm)

# Create the input for the assignment agent
input_dict = {
    "projects": projects_str,
    "candidates": candidates_str
}

# Run the assignment agent
assignment_output = assignment_chain.run(input_dict)
assignment_output

'```json\n[\n    {\n        "Project": "Project 1: Equity Valuation Model",\n        "Assignments": {\n            "Q1": ["Jack", "Susan"],\n            "Q2": ["Jack", "Maria"],\n            "Q3": ["Tom", "Susan"],\n            "Q4": ["Judy", "Jack"]\n        }\n    },\n    {\n        "Project": "Project 2: Algorithmic Trading Bot",\n        "Assignments": {\n            "Q1": ["Alex", "Jane"],\n            "Q2": ["Alex", "Maria"],\n            "Q3": ["Alex", "Jane"],\n            "Q4": ["Alex", "Jane"]\n        }\n    },\n    {\n        "Project": "Project 3: Sentiment Analysis for Stock Predictions",\n        "Assignments": {\n            "Q1": ["Jack", "Susan"],\n            "Q2": ["Jack", "Maria"],\n            "Q3": ["Tom", "Susan"],\n            "Q4": ["Judy", "Jack"]\n        }\n    },\n    {\n        "Project": "Project 4: Portfolio Optimization Dashboard",\n        "Assignments": {\n            "Q1": ["Alex", "Jane"],\n            "Q2": ["Alex", "Maria"],\n            "Q3": ["

In [90]:
# Extract JSON from the output
def extract_json(text):
    json_regex = re.compile(r'\[\s*\{.*?\}\s*\]', re.DOTALL)
    match = json_regex.search(text)
    if match:
        return match.group(0)
    else:
        return None

# Use the extract_json function
json_text = extract_json(assignment_output)

if json_text is None:
    print("Failed to extract JSON from the LLM output.")
    assignments = []
else:
    try:
        assignments = json.loads(json_text)
    except json.JSONDecodeError as e:
        print("Failed to parse assignment output:", e)
        assignments = []

assignments

[{'Project': 'Project 1: Equity Valuation Model',
  'Assignments': {'Q1': ['Jack', 'Susan'],
   'Q2': ['Jack', 'Maria'],
   'Q3': ['Tom', 'Susan'],
   'Q4': ['Judy', 'Jack']}},
 {'Project': 'Project 2: Algorithmic Trading Bot',
  'Assignments': {'Q1': ['Alex', 'Jane'],
   'Q2': ['Alex', 'Maria'],
   'Q3': ['Alex', 'Jane'],
   'Q4': ['Alex', 'Jane']}},
 {'Project': 'Project 3: Sentiment Analysis for Stock Predictions',
  'Assignments': {'Q1': ['Jack', 'Susan'],
   'Q2': ['Jack', 'Maria'],
   'Q3': ['Tom', 'Susan'],
   'Q4': ['Judy', 'Jack']}},
 {'Project': 'Project 4: Portfolio Optimization Dashboard',
  'Assignments': {'Q1': ['Alex', 'Jane'],
   'Q2': ['Alex', 'Maria'],
   'Q3': ['Alex', 'Maria'],
   'Q4': ['Alex', 'Jane']}},
 {'Project': 'Project 5: Risk Management Simulation',
  'Assignments': {'Q1': ['Jack', 'Susan'],
   'Q2': ['Jack', 'Maria'],
   'Q3': ['Tom', 'Susan'],
   'Q4': ['Judy', 'Jack']}}]

In [91]:
# Transform data to fit the desired DataFrame structure
rows = []
for item in assignments:
    row = {'Project': item['Project']}
    for quarter, names in item['Assignments'].items():
        row[quarter] = ', '.join(names)  # Convert list of names to a comma-separated string
    rows.append(row)

# Create DataFrame
df = pd.DataFrame(rows, columns=['Project', 'Q1', 'Q2', 'Q3', 'Q4'])
df

,Project,Q1,Q2,Q3,Q4
0,Project 1: Equity Valuation Model,"Jack, Susan","Jack, Maria","Tom, Susan","Judy, Jack"
1,Project 2: Algorithmic Trading Bot,"Alex, Jane","Alex, Maria","Alex, Jane","Alex, Jane"
2,Project 3: Sentiment Analysis for Stock Predic...,"Jack, Susan","Jack, Maria","Tom, Susan","Judy, Jack"
3,Project 4: Portfolio Optimization Dashboard,"Alex, Jane","Alex, Maria","Alex, Maria","Alex, Jane"
4,Project 5: Risk Management Simulation,"Jack, Susan","Jack, Maria","Tom, Susan","Judy, Jack"


## 1. Langgraph - manager only - BL on

In [ ]:
from typing import Annotated, List, Tuple, Union
from langchain.tools import BaseTool, StructuredTool, Tool
from langchain_experimental.tools import PythonREPLTool
from langchain_core.tools import tool
import random
import numpy as np
import matplotlib.pyplot as plt

# This executes code locally, which can be unsafe
python_repl_tool = PythonREPLTool()

In [ ]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI

def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    # Each worker node will be given a name and some tools.
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor
  # agent node
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

In [ ]:
@tools
def analyze_tool(input_dict:dict):
  '''
  Initiate or update the project plan.
  input_dict = {
      "projects": projects_str,
      "candidates": candidates_str
  }
  '''
  # Adjust the assignment prompt template
  assignment_prompt_template = PromptTemplate(
      input_variables=["projects", "candidates"],
      template="""
  You are the Assignment Agent responsible for assigning candidates to projects across all quarters.

  Projects:
  {projects}

  Candidates:
  {candidates}

  Constraints:
  - Ensure that each candidate is not assigned beyond their capacity for each quarter.
  - Candidates must be available in the quarter they are assigned.
  - Match candidate skills and roles with project requirements.
  - Consider performance reviews when selecting candidates.

  Your task is to assign the best candidates to each project for each quarter.

  Provide the assignments in the following JSON format:

  [
      {{
          "Project": "Project Name",
          "Assignments": {{
              "Q1": ["Candidate Name 1", "Candidate Name 2"],
              "Q2": ["Candidate Name 3"],
              "Q3": [],
              "Q4": ["Candidate Name 4"]
          }}
      }},
      ...
  ]

  Ensure the JSON is valid and parsable.
  """
  )

  llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=openai.api_key)
  assignment_chain = LLMChain(prompt=assignment_prompt_template, llm=llm)
  # Create the input for the assignment agent

  # Run the assignment agent
  assignment_output = assignment_chain.run(input_dict)


first_agent = create_agent(llm,
                      tools = [search_project],
                      system_prompt = project_prompt)
project_node = functools.partial(agent_node, agent=project_agent, name="project_agent")

Ignore below


In [ ]:
import json

# Initialize the list to hold allocations
optimized_allocations = []

try:
    assignments = json.loads(assignment_output)
except json.JSONDecodeError as e:
    print("Failed to parse assignment output:", e)
    assignments = []

# Process the assignments
for assignment in assignments:
    project_name = assignment['Project']
    project_allocation = {'Project': project_name}
    for quarter in quarters:
        assigned_candidates = assignment['Assignments'].get(quarter, [])
        for candidate_name in assigned_candidates:
            # Find the candidate in people_data
            if candidate_name in people_data['Name'].values:
                person_idx = people_data[people_data['Name'] == candidate_name].index[0]
                # Check capacity and availability
                if people_data.at[person_idx, f'{quarter}_available'] and people_data.at[person_idx, f'{quarter}_capacity'] > 0:
                    # Update capacity and availability
                    people_data.at[person_idx, f'{quarter}_capacity'] -= 1
                    if people_data.at[person_idx, f'{quarter}_capacity'] <= 0:
                        people_data.at[person_idx, f'{quarter}_available'] = False
                else:
                    # Candidate not available or over capacity
                    assigned_candidates.remove(candidate_name)
                    print(f"{candidate_name} is not available or over capacity in {quarter}.")
            else:
                # Candidate not found
                assigned_candidates.remove(candidate_name)
                print(f"{candidate_name} not found in candidate list.")

        # Update the project allocation for the quarter
        project_allocation[quarter] = ', '.join(assigned_candidates) if assigned_candidates else "No assignment"

    optimized_allocations.append(project_allocation)


Failed to parse assignment output: Expecting value: line 1 column 1 (char 0)


In [ ]:
optimized_allocations

[]

In [ ]:
# Function to format candidates for the prompt
def format_candidates(candidates):
    candidates_str = ""
    for candidate in candidates:
        candidates_str += f"Name: {candidate['Name']}, Role: {candidate['Role']}, Skills: {candidate['Skills']}, Performance Review: {candidate['Performance Review']}, Capacity: {candidate['Capacity']}\n"
    return candidates_str

# Prompt template for LLM chain
prompt_template = PromptTemplate(
    input_variables=["project", "candidates", "constraints"],
    template="""
Project requirements:
{project}

Candidates:
{candidates}

Additional constraints:
{constraints}

Recommend the best candidate for this project, considering each candidate’s capacity, performance, and availability.
Provide only the name of the selected candidate.
"""
)

# Initialize LLM Chain
llm = ChatOpenAI(model="gpt-4", openai_api_key=openai.api_key)
llm_chain = LLMChain(prompt=prompt_template, llm=llm)

<ipython-input-18-2c7d31907d82>:27: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4", openai_api_key=openai.api_key)
<ipython-input-18-2c7d31907d82>:28: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt_template, llm=llm)


In [ ]:
ptimized_allocations = []

# Initialize unassigned candidates list per quarter
unassigned_candidates = {quarter: people_data[people_data[f'{quarter}_available']].copy() for quarter in quarters}

# Calculate total available candidates per quarter
total_available_candidates_per_quarter = {quarter: len(unassigned_candidates[quarter]) for quarter in quarters}

# Determine total projects
total_projects = len(project_data)

for _, project in project_data.iterrows():
    project_description = project['Description']
    constraints = project.get('constraints', 'No constraints')  # Add any project-specific constraints
    project_name = project['name']

    project_allocation = {'Project': project_name}

    for quarter in quarters:
          # Get unassigned candidates for the quarter
        candidates_df = unassigned_candidates[quarter]
        if candidates_df.empty:
            assigned_candidates = f"No available candidates for {quarter}"
        else:
            # Retrieve similar candidates for the project and quarter
            candidates_list = retrieve_similar_people(project_description, quarter)
            if not candidates_list:
                assigned_candidates = f"No available candidates for {quarter}"
            else:
                # Limit candidates to those who are still unassigned
                candidate_names_unassigned = set(candidates_df['Name'])
                candidates_list = [c for c in candidates_list if c['Name'] in candidate_names_unassigned]

                if not candidates_list:
                    assigned_candidates = f"No available candidates for {quarter}"
                else:
                    # Determine number of required candidates
                    num_required = project.get('NumRequired', None)
                    if num_required is None:
                        # Distribute candidates evenly among projects
                        total_candidates = total_available_candidates_per_quarter[quarter]
                        num_required = max(1, total_candidates // total_projects)

                    # Adjust num_required if there are not enough candidates left
                    num_required = min(num_required, len(candidates_list))

                    # Limit candidates list to the number required
                    candidates_to_assign = candidates_list[:num_required]

                    # Format candidates
                    candidates_str = format_candidates(candidates_to_assign)

                    input_dict = {
                        "project": project_description,
                        "candidates": candidates_str,
                        "constraints": constraints,
                        "num_required": num_required
                    }

                    # Run assignment through the llm_chain
                    allocation = llm_chain.run(input_dict)

                    # Extract assigned candidates' names
                    assigned_candidates = allocation.strip()

                    # Split and clean assigned candidates' names
                    assigned_candidate_names = [name.strip() for name in assigned_candidates.split(',')]

                    # Verify that assigned candidates are among the unassigned candidates
                    valid_assigned_candidates = []
                    for name in assigned_candidate_names:
                        if name in candidate_names_unassigned:
                            valid_assigned_candidates.append(name)
                            # Update capacity and availability
                            person_idx = people_data[people_data['Name'] == name].index[0]
                            people_data.at[person_idx, f'{quarter}_capacity'] -= 1
                            if people_data.at[person_idx, f'{quarter}_capacity'] <= 0:
                                people_data.at[person_idx, f'{quarter}_available'] = False
                            # Remove from unassigned candidates
                            unassigned_candidates[quarter] = unassigned_candidates[quarter][unassigned_candidates[quarter]['Name'] != name]
                        else:
                            # Candidate not valid or already assigned
                            pass
                    assigned_candidates = ', '.join(valid_assigned_candidates)
                    # Update total available candidates
                    total_available_candidates_per_quarter[quarter] -= len(valid_assigned_candidates)

        # Update project allocation for the quarter
        project_allocation[quarter] = assigned_candidates

    # Append project allocation to optimized_allocations
    optimized_allocations.append(project_allocation)

# Convert results to a DataFrame for easy saving
final_allocation = pd.DataFrame(optimized_allocations)

In [ ]:
final_allocation

,Project,Q1,Q2,Q3,Q4
0,**Project 1: Equity Valuation Model**,Alice,Alice,Alice,Alice
1,**Project 2: Algorithmic Trading Bot**,Judy,Judy,Alice,Alice
2,**Project 3: Sentiment Analysis for Stock Pred...,Alice,Alice,Alice,Judy
3,**Project 4: Portfolio Optimization Dashboard**,Alice,Alice,Tom,Alice
4,**Project 5: Risk Management Simulation**,Kyle,Kyle,Kyle,Kyle
5,**Project 1: Equity Valuation Model**,Tom,Tom,Tom,Tom
6,**Project 2: Algorithmic Trading Bot**,Alex,Alex,Alex,Alex
7,**Project 3: Sentiment Analysis for Stock Pred...,Kyle,Kyle,Kyle,Kyle
8,**Project 4: Portfolio Optimization Dashboard**,Jack,Jack,Jack,Jack
9,**Project 5: Risk Management Simulation**,Maria,Maria,Susan,Maria


In [ ]:
# Define prompts for agents
assignment_prompt_template = PromptTemplate(
    input_variables=["project", "candidates", "constraints"],
    template="""
Project requirements:
{project}

Candidates:
{candidates}

Constraints:
{constraints}

As the Assignment Agent, recommend the best candidate(s) for this project, considering each candidate’s capacity, performance, and availability.
Provide only the name(s) of the selected candidate(s).
"""
)

reviewer_prompt_template = PromptTemplate(
    input_variables=["project", "assignment", "constraints"],
    template="""
Project requirements:
{project}

Proposed Assignment:
{assignment}

Constraints:
{constraints}

As the Reviewer Agent, check the proposed assignment against the constraints.
If it complies, confirm the assignment.
If not, suggest necessary changes to meet the constraints.
"""
)

In [ ]:
# Initialize LLM Chains for agents
assignment_chain = LLMChain(prompt=assignment_prompt_template, llm=llm)
reviewer_chain = LLMChain(prompt=reviewer_prompt_template, llm=llm)

# Initialize agents
# For simplicity, we'll define agent functions instead of using LangChain agents

def retrieval_agent(project_description, quarter):
    candidates = retrieve_similar_people(project_description, quarter)
    return candidates

def assignment_agent(project_description, candidates, constraints):
    candidates_str = format_candidates(candidates)
    input_dict = {
        "project": project_description,
        "candidates": candidates_str,
        "constraints": constraints
    }
    assignment = assignment_chain.run(input_dict)
    return assignment.strip()

def reviewer_agent(project_description, assignment, constraints):
    input_dict = {
        "project": project_description,
        "assignment": assignment,
        "constraints": constraints
    }
    review = reviewer_chain.run(input_dict)
    return review.strip()

# Main allocation process
optimized_allocations = []

for _, project in project_data.iterrows():
    project_description = project['Description']
    constraints = project.get('constraints', 'should have at least 2 candidates')
    project_name = project['name']

    project_allocation = {'Project': project_name}

    for quarter in quarters:
        # Step 1: Retrieval Agent retrieves candidates
        candidates = retrieval_agent(project_description, quarter)

        if not candidates:
            assigned_candidate = f"No available candidates for {quarter}"
        else:
            # Step 2: Assignment Agent proposes an assignment
            assignment = assignment_agent(project_description, candidates, constraints)

            # Step 3: Reviewer Agent checks the assignment
            review = reviewer_agent(project_description, assignment, constraints)

            # Interpret the reviewer's response
            if "confirm" in review.lower():
                assigned_candidate = assignment
                # Update capacity and availability
                assigned_names = [name.strip() for name in assigned_candidate.split(',')]
                for name in assigned_names:
                    if name in people_data['Name'].values:
                        person_idx = people_data[people_data['Name'] == name].index[0]
                        people_data.at[person_idx, f'{quarter}_capacity'] -= 1
                        if people_data.at[person_idx, f'{quarter}_capacity'] <= 0:
                            people_data.at[person_idx, f'{quarter}_available'] = False
            else:
                # If the reviewer suggests changes, we could implement them
                # For simplicity, we'll assign as per reviewer's suggestion
                # Extract suggested assignment from the review
                suggested_assignment = review.split("suggestion:")[-1].strip()
                assigned_candidate = suggested_assignment
                # Update capacity and availability
                assigned_names = [name.strip() for name in assigned_candidate.split(',')]
                for name in assigned_names:
                    if name in people_data['Name'].values:
                        person_idx = people_data[people_data['Name'] == name].index[0]
                        people_data.at[person_idx, f'{quarter}_capacity'] -= 1
                        if people_data.at[person_idx, f'{quarter}_capacity'] <= 0:
                            people_data.at[person_idx, f'{quarter}_available'] = False

        # Update project allocation for the quarter
        project_allocation[quarter] = assigned_candidate

    # Append project allocation to optimized_allocations
    optimized_allocations.append(project_allocation)

# Convert results to a DataFrame for easy saving
final_allocation = pd.DataFrame(optimized_allocations)

In [ ]:
final_allocation['Q1'][3]

'The proposed assignment does not meet the constraints. The constraint states there should be at least 2 candidates, but only Jack and Jane are mentioned. Please add at least one more candidate to meet the constraint.'

In [ ]:
final_allocation['Q1']

,Q1
0,The proposed assignment complies with the cons...
1,"Alex, Jane"
2,"Tom, Alex"
3,The proposed assignment does not meet the cons...
4,The proposed assignment complies with the cons...


Candidate Retrieval Tool

In [ ]:
def retrieve_candidates_tool_func(input):
    import re
    match = re.match(r"Retrieve candidates for the project: (.*) in quarter (\w+)\.", input)
    if match:
        project_description = match.group(1)
        quarter = match.group(2)
        candidates = retrieve_similar_people(project_description, quarter)
        return str(candidates)
    else:
        return "Error: Could not parse input."

retrieve_candidates_tool = Tool(
    name="RetrieveCandidates",
    func=retrieve_candidates_tool_func,
    description="Retrieve potential candidates for a project based on the project description."
)


Constraint Evaluation Tool

In [ ]:
def evaluate_constraints_tool_func(input):
    import re
    match = re.match(r"Evaluate constraints for the candidates: (.*), constraints: (.*), quarter: (\w+)\.", input)
    if match:
        candidates_str = match.group(1)
        constraints = match.group(2)
        quarter = match.group(3)
        import ast
        try:
            candidates = ast.literal_eval(candidates_str)
        except Exception as e:
            return f"Error parsing candidates: {e}"
        # Here you can apply actual constraint evaluation logic
        return str(candidates)
    else:
        return "Error: Could not parse input."

evaluate_constraints_tool = Tool(
    name="EvaluateConstraints",
    func=evaluate_constraints_tool_func,
    description="Filter candidates based on constraints like availability, capacity, and performance."
)

Assignment Tool

In [ ]:
def assign_candidate_tool_func(input):
    import re
    match = re.match(r"Assign the best candidate for the project: (.*) from candidates: (.*)\.", input)
    if match:
        project_description = match.group(1)
        candidates_str = match.group(2)
        import ast
        try:
            candidates = ast.literal_eval(candidates_str)
        except Exception as e:
            return f"Error parsing candidates: {e}"
        candidates_str_formatted = format_candidates(candidates)
        input_dict = {
            "project": project_description,
            "candidates": candidates_str_formatted,
            "constraints": "Consider capacity, performance, and availability."
        }
        allocation = llm_chain.run(input_dict)
        return allocation.strip()
    else:
        return "Error: Could not parse input."

assign_candidate_tool = Tool(
    name="AssignCandidate",
    func=assign_candidate_tool_func,
    description="Assign the best candidate to the project."
)

Feedback Tool (Optional)

In [ ]:
# Candidate Retrieval Agent
from langchain.agents import AgentType

candidate_retrieval_agent = initialize_agent(
    tools=[retrieve_candidates_tool],
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent_kwargs={"prefix": "You are an agent that retrieves candidates."},
    handle_parsing_errors=True
)

# Similarly for the other agents


In [ ]:
# Constraint Evaluation Agent
constraint_evaluation_agent = initialize_agent(
    tools=[evaluate_constraints_tool],
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent_kwargs={"prefix": "You are an agent that evaluates constraints."},
    handle_parsing_errors=True
)

In [ ]:
# Assignment Agent
assignment_agent = initialize_agent(
    tools=[assign_candidate_tool],
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent_kwargs={"prefix": "You are an agent that assigns the best candidate to the project."},
    handle_parsing_errors=True
)

Set Up Shared Memory

In [ ]:
# Set Up Shared Memory
shared_memory = ConversationBufferMemory(memory_key="chat_history")

# Assign memory to agents
candidate_retrieval_agent.memory = shared_memory
constraint_evaluation_agent.memory = shared_memory
assignment_agent.memory = shared_memory

In [ ]:
def orchestrate_agents(project, quarter):
    project_description = project['Description']
    constraints = project.get('constraints', 'No constraints')

    # Step 1: Candidate Retrieval
    retrieval_input = f"Retrieve candidates for the project: {project_description} in quarter {quarter}."
    candidates = candidate_retrieval_agent.run(retrieval_input)

    # Parse candidates
    if isinstance(candidates, str):
        import ast
        try:
            candidates = ast.literal_eval(candidates)
        except Exception as e:
            print(f"Error parsing candidates: {e}")
            candidates = []

    # Step 2: Constraint Evaluation
    if candidates:
        evaluation_input = f"Evaluate constraints for the candidates: {candidates}, constraints: {constraints}, quarter: {quarter}."
        filtered_candidates = constraint_evaluation_agent.run(evaluation_input)
    else:
        filtered_candidates = []

    # Parse filtered candidates
    if isinstance(filtered_candidates, str):
        import ast
        try:
            filtered_candidates = ast.literal_eval(filtered_candidates)
        except Exception as e:
            print(f"Error parsing filtered candidates: {e}")
            filtered_candidates = []

    # Step 3: Assignment
    if filtered_candidates:
        assignment_input = f"Assign the best candidate for the project: {project_description} from candidates: {filtered_candidates}."
        assigned_candidate = assignment_agent.run(assignment_input).strip()
    else:
        assigned_candidate = f"No available candidates for {quarter}"

    # Update capacities and availability
    if assigned_candidate in people_data['Name'].values:
        person_idx = people_data[people_data['Name'] == assigned_candidate].index[0]
        people_data.at[person_idx, f'{quarter}_capacity'] -= 1
        if people_data.at[person_idx, f'{quarter}_capacity'] <= 0:
            people_data.at[person_idx, f'{quarter}_available'] = False

    return assigned_candidate


In [ ]:
# Main Loop
optimized_allocations = []

for _, project in project_data.iterrows():
    project_name = project['name']
    project_allocation = {'Project': project_name}

    for quarter in quarters:
        assigned_candidate = orchestrate_agents(project, quarter)
        project_allocation[quarter] = assigned_candidate

    optimized_allocations.append(project_allocation)

# Convert results to a DataFrame
final_allocation = pd.DataFrame(optimized_allocations)

Error parsing candidates: unterminated string literal (detected at line 1) (<unknown>, line 1)
Error parsing candidates: invalid syntax (<unknown>, line 1)


KeyboardInterrupt: 

In [ ]:
optimized_allocations = []

for _, project in project_data.iterrows():
    project_description = project['Description']
    constraints = project.get('constraints', 'No constraints')  # Add any project-specific constraints
    project_name = project['name']

    project_allocation = {'Project': project_name}

    for quarter in quarters:
        # Retrieve similar candidates for the project and quarter
        candidates = retrieve_similar_people(project_description, quarter)

        if not candidates:
            assigned_candidate = f"No available candidates for {quarter}"
        else:
            # Format candidates
            candidates_str = format_candidates(candidates)

            input_dict = {
                "project": project_description,
                "candidates": candidates_str,
                "constraints": constraints
            }

            # Run assignment through the llm_chain
            allocation = llm_chain.run(input_dict)

            # Extract assigned candidate's name
            assigned_candidate = allocation.strip()

            # Verify that assigned_candidate is among the candidates
            candidate_names = [c['Name'] for c in candidates]
            if assigned_candidate not in candidate_names:
                assigned_candidate = f"No valid assignment for {quarter}"
            else:
                # Update capacity and availability
                person_idx = people_data[people_data['Name'] == assigned_candidate].index[0]
                people_data.at[person_idx, f'{quarter}_capacity'] -= 1
                if people_data.at[person_idx, f'{quarter}_capacity'] <= 0:
                    people_data.at[person_idx, f'{quarter}_available'] = False

        # Update project allocation for the quarter
        project_allocation[quarter] = assigned_candidate

    # Append project allocation to optimized_allocations
    optimized_allocations.append(project_allocation)

# Convert results to a DataFrame for easy saving
final_allocation = pd.DataFrame(optimized_allocations)


In [ ]:
final_allocation

,Project,Q1,Q2,Q3,Q4
0,**Project 1: Equity Valuation Model**,Alice,Alice,Alice,Alice
1,**Project 2: Algorithmic Trading Bot**,Judy,Judy,Alice,Judy
2,**Project 3: Sentiment Analysis for Stock Pred...,Alice,Alice,Judy,Alice
3,**Project 4: Portfolio Optimization Dashboard**,Alice,Alice,Alice,Alice
4,**Project 5: Risk Management Simulation**,Kyle,Kyle,Kyle,Kyle
